<a href="https://colab.research.google.com/github/neha33-pro/Task2Crypto/blob/main/Task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ---------------------------------------------------------
#   Hybrid Cryptographic File Backup System
#   AES encryption + RSA key wrapping + Digital signature
#   Supports two independent users, each able to decrypt
# ---------------------------------------------------------

!pip install cryptography

from cryptography.hazmat.primitives.asymmetric import rsa, padding as asym_padding
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding as sym_padding
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.exceptions import InvalidSignature

import os, random
from google.colab import files

# ---------------------------------------------------------
#   Shows upload file button e.g  Report.pdf
# ---------------------------------------------------------
files.upload()     # user selects file

with open("Report.pdf", "rb") as f:
    plaintext = f.read()

print("File loaded. Size:", len(plaintext), "bytes")


# -----------------------------------------------------------
#  Random selection of key size and public key exponents
#  Generates two RSA public and private keys for two users
# -----------------------------------------------------------
public_exponents = [3, 65537]
public_exp = random.choice(public_exponents)
key_sizes = [2048, 1024]
key_size = random.choice(key_sizes)

# User 1 keypair
user1_private_key = rsa.generate_private_key(
    public_exponent=public_exp,
    key_size=key_size,
    backend=default_backend()
)
user1_public_key = user1_private_key.public_key()

# User 2 keypair
user2_private_key = rsa.generate_private_key(
    public_exponent=public_exp,
    key_size=key_size,
    backend=default_backend()
)
user2_public_key = user2_private_key.public_key()


# ---------------------------------------------------------
#   Generate AES key and IV
# ---------------------------------------------------------
aes_key = os.urandom(32)         # AES-256 key (32 bytes)
iv = os.urandom(16)              # AES CBC requires a 16-byte IV


# --------------------------------------------------------------
#   Get password from user and derive key from it using(PBKDF2)
# --------------------------------------------------------------
password = input("Enter password for secure storage: ").encode()

salt = os.urandom(16)            # random salt for PBKDF2

kdf = PBKDF2HMAC(
    algorithm=hashes.SHA256(),
    length=32,
    salt=salt,
    iterations=100_000,
    backend=default_backend()
)

# --------------------------------------------------------
#  Transform user password into strong cryptography key
# --------------------------------------------------------

password_key = kdf.derive(password)

# ---------------------------------------------------------
#   Encrypt the file using AES-CBC
# ---------------------------------------------------------

# 1) Pad plaintext to AES block size
padder = sym_padding.PKCS7(128).padder()
padded_plaintext = padder.update(plaintext) + padder.finalize()

# 2) Perform AES encryption
cipher = Cipher(algorithms.AES(aes_key), modes.CBC(iv), backend=default_backend())
encryptor = cipher.encryptor()
ciphertext = encryptor.update(padded_plaintext) + encryptor.finalize()

print("File encrypted with AES-CBC.")


# -----------------------------------------------
#   Sign the ciphertext with user1 private key
# -----------------------------------------------
signature = user1_private_key.sign(
    ciphertext,
    asym_padding.PSS(
        mgf=asym_padding.MGF1(hashes.SHA256()),
        salt_length=asym_padding.PSS.MAX_LENGTH
    ),
    hashes.SHA256()
)

with open("Report_signature.sig", "wb") as f:
    f.write(signature)

print("Digital signature saved.")


# ---------------------------------------------------------
#  Encrypt AES key of user1 and user2 using RSA public key
# ---------------------------------------------------------
encrypted_key_user1 = user1_public_key.encrypt(
    aes_key,
    asym_padding.OAEP(
        mgf=asym_padding.MGF1(hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)

encrypted_key_user2 = user2_public_key.encrypt(
    aes_key,
    asym_padding.OAEP(
        mgf=asym_padding.MGF1(hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)


# ---------------------------------------------------------
#   Save all components needed for future decryption
# ---------------------------------------------------------
with open("Encrypted_Report.pdf", "wb") as f:
    f.write(ciphertext)

with open("iv.bin", "wb") as f:
    f.write(iv)

with open("salt.bin", "wb") as f:
    f.write(salt)

with open("Encrypted_AES_Key_User1.bin", "wb") as f:
    f.write(encrypted_key_user1)

with open("Encrypted_AES_Key_User2.bin", "wb") as f:
    f.write(encrypted_key_user2)

print("Encrypted file, RSA-wrapped AES keys, salt, and IV saved.")


# =========================================================
#               DECRYPTION SECTION
# =========================================================

print("\n=== Decryption Phase ===")

# ---------------------------------------------------------
#   Load everything back from disk
# ---------------------------------------------------------
with open("Encrypted_Report.pdf", "rb") as f:
    stored_ciphertext = f.read()

with open("Report_signature.sig", "rb") as f:
    stored_signature = f.read()

with open("iv.bin", "rb") as f:
    stored_iv = f.read()

with open("salt.bin", "rb") as f:
    stored_salt = f.read()

with open("Encrypted_AES_Key_User1.bin", "rb") as f:
    stored_wrapped1 = f.read()

with open("Encrypted_AES_Key_User2.bin", "rb") as f:
    stored_wrapped2 = f.read()


# ---------------------------------------------------------
#   Verify digital signature before decrypting
# ---------------------------------------------------------
try:
    user1_public_key.verify(
        stored_signature,
        stored_ciphertext,
        asym_padding.PSS(
            mgf=asym_padding.MGF1(hashes.SHA256()),
            salt_length=asym_padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )
    print("Signature verified. Ciphertext is authentic.")
except InvalidSignature:
    raise SystemExit("Signature verification failed — file may be tampered.")


# ---------------------------------------------------------
#   Ask which user will decrypt
# ---------------------------------------------------------
choice = input("Decrypt as which user? (1 or 2): ").strip()


if choice == "1":
    chosen_encrypted_key = stored_wrapped1
    chosen_private_key = user1_private_key
    output_filename = "Decrypted_by_User1.pdf"
elif choice == "2":
    chosen_wrapped_key = stored_wrapped2
    chosen_private_key = user2_private_key
    output_filename = "Decrypted_by_User2.pdf"
else:
    raise SystemExit("Invalid user selected.")


# ---------------------------------------------------------
#  Decrypt AES key using RSA private key for chosen user
# ---------------------------------------------------------
recovered_aes_key = chosen_private_key.decrypt(
    chosen_encrypted_key,
    asym_padding.OAEP(
        mgf=asym_padding.MGF1(hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)


# ---------------------------------------------------------
#   AES decrypt ciphertext using recovered AES key
# ---------------------------------------------------------
decrypt_cipher = Cipher(
    algorithms.AES(recovered_aes_key),
    modes.CBC(stored_iv),
    backend=default_backend()
)
decryptor = decrypt_cipher.decryptor()
decrypted_padded = decryptor.update(stored_ciphertext) + decryptor.finalize()

# Remove PKCS7 padding
unpadder = sym_padding.PKCS7(128).unpadder()
decrypted_plaintext = unpadder.update(decrypted_padded) + unpadder.finalize()

# Save output
with open(output_filename, "wb") as f:
    f.write(decrypted_plaintext)

print(f"Decryption completed. Output saved as: {output_filename}")


Saving Report.pdf to Report.pdf
File loaded. Size: 57998 bytes
Enter password for secure storage: MyStrongPassword
File encrypted with AES-CBC.
Digital signature saved.
Encrypted file, RSA-wrapped AES keys, salt, and IV saved.

=== Decryption Phase ===
Signature verified. Ciphertext is authentic.
Decrypt as which user? (1 or 2): 2
Decryption completed. Output saved as: Decrypted_by_User2.pdf
